* 时间戳，具体的时刻
* 固定的时间区间，例如2007的1月或整个2010年
* 时间间隔，由开始和结束时间戳表示。时间区间可以被认为是间隔的特殊情况
* 实验时间或消耗时间。每个时间戳是相对于特定开始时间的时间的量度

##  1.日期和时间数据的类型及工具

Python标准库包含了日期和时间数据的类型，也包括日历相关的功能  
datetime.datetime和calender  

__字符串和datetime互相转换__

使用str方法或传递一个指定的格式给strftime方法来对datetime对象和pandas的Timestamp对象进行格式化

In [ ]:
from datetime import datetime
stamp = datetime(2011, 1, 3)

In [ ]:
str(stamp)

In [ ]:
stamp.strftime('%Y-%m-%d')

也可以使用strptime和格式代码，将字符串转化日期

In [ ]:
value = '2011-01-03'
datetime.strptime(value, '%Y-%m-%d')

直接使用parser.parse方法解析大部分人类可理解的日期表示

In [ ]:
from dateutil.parser import parse
parse('2011-01-03')
parse('6/12/2011', dayfirst=True)

pd.to_datetime方法可以转换很多不同的日期表示格式

In [ ]:
pd.to_datetime(datestrs)

##  2.时间序列基础

pandas中的基础时间序列种类是有时间戳索引的Series，在pandas外部则通常表示为Python字符串或datetime对象

自己理解：datetime类型被作为Series的索引，就变成了DatetimeIndex,DatetimeIndex中的标量值就是pandas中的TimeStamp对象

__索引、选择、子集__

In [ ]:
stamp = ts.index[2]    #基于标签进行索引和选择
ts[stamp]

In [ ]:
longer_ts['2001']    #传递一个年份选择数据切片，这里，字符串'2001'被解释为一个年份，并选择了相应的时间区间。

In [ ]:
ts[datetime(2011, 1, 7)]    #使用datetime对象进行切片

__含有重复索引的时间序列__

In [ ]:
dup_ts.index.is_unique    #查看索引是否唯一

grouped = dup_ts.groupby(level=0)    #聚合含有非唯一时间戳的数据
grouped.mean()

##  3.日期范围、频率和移位

pandas的通用时间序列是不规则的，即时间序列的频率不是固定的  
可以通过调用resample方法将样本时间序列转换为固定的每日频率数据

In [ ]:
resample = ts.resample('D')    #'D'被解释为每日频率

__生成日期范围__

In [ ]:
index = pd.date_range('2012-04-01', '2012-06-01')    #pd.date_range用于根据特定频率生成指定长度的DatetimeIndex
#默认情况下，生成的是每日的时间戳。

In [ ]:
pd.date_range(start='2012-04-01', periods=20)  #如果只传递一个起始或结尾日期，必须传递一个用于生成范围的数字 

In [ ]:
pd.date_range('2000-01-01', '2000-12-01', freq='BM')    #传递频率参数。  此例中生成结果不包括最前的和最后的时间戳

In [ ]:
pd.date_range('2012-05-02 12:56:31', periods=5, normalize=True)    #生成标准化为0的时间戳

__频率和日期偏置__

In [ ]:
pd.date_range('2000-01-01', periods=10, freq='1h30min')    #对于每个基础频率，都有一个对象可以被用于定义日期偏置

__移位（前向和后向）日期__

In [ ]:
ts.shift(2)    #Series和DataFrame都有一个shift方法用于进行简单的前向和后向移位，而不改变索引

In [ ]:
ts / ts.shift(1) - 1   #最常见的用法，计算时间序列或DataFrame多列时间序列的百分比变化

In [ ]:
ts.shift(2, freq='M')#简单移位并不改变索引，一些数据会被丢弃。因此，如果频率是已知的，则可以将频率传递给shift来推移时间戳而不是简单的数据
                    #其他的频率也可以传递

使用偏置进行移位日期： pandas日期偏置也可以使用datetime或Timestamp对象完成

In [ ]:
offset = MonthEnd()
offset.rollforward(now)    #锚定偏置可以显式地将日期向前或向后“滚动”
offset.rollback(now)

In [ ]:
ts.groupby(offset.rollforward).mean()    #创造性用法

##  4.时区处理

目前用不到

##  5.时间区间和区间算术

时间区间表示的是时间范围，比如一些天、一些月、一些季度或者是一些年。  
period类表示的正式这种数据类型

In [ ]:
p  = pd.Period(2007, freq='A-DEC')    #在这个例子中，Period对象表示的是从2007年1月1日到2007年12月31日（包含在内）的时间段

In [ ]:
rng = pd.period_range('2000-01-01', '2000-06-30', freq='M')    #使用period_range函数可以构造规则区间序列，生成PeriodIndex对象

In [ ]:
pd.Series(np.random.randn(6), index=rng)    #PeriodIndex类存储的是区间的序列，可以作为任意pandas数据结构的轴索引

In [ ]:
index = pd.PeriodIndex(values, freq='Q-DEC')    #values是个字符串列表，可以直接使用PeriodIndex类

__区间频率转换__

In [ ]:
p = pd.Period('2007', freq='A-DEC')

p.asfreq("M", how='start')

__季度区间频率__

In [ ]:
p = pd.Period('2012Q4', freq='Q-JAN')

p.asfreq('D', how='start')

__将时间戳转换为区间（以及逆转换）__

通过时间戳索引的Series和DataFrame可以被to_period方法转换为区间

In [ ]:
rng = pd.date_range('2000-01-01', periods=3, freq='M')

ts = pd.Series(np.random.randn(3), index=rng)

pts = ts.to_period()

使用to_timestamp可以将区间再转换为时间戳

In [ ]:
pts.to_timestamp(how='end')

__从数组生成PeriodIndex__

In [ ]:
index = pd.PeriodIndex(year=data.year, quarter=data.quarter, freq='Q-DEC')

data.index = index

##  6.  重新采样与频率转换

pandas对象都配有resample方法，该方法是所有频率转换的工具函数。resample拥有类似于groupby的API，调用resample对数据分组，之后再调用聚合函数。  
将更高频率的数据聚合到低频率被称为向下采样，而从低频率转换到高频率称为向上采样

In [ ]:
rng = pd.date_range('2000-01-01', periods=100, freq="D")

ts = pd,Series(np.random.randn(len(rng)), index=rng)

In [ ]:
ts.resample("M").mean()    #pandas对象都配有resample方法，该方法是所有频率转换的工具函数。
                            #resample拥有类似于groupby的API，调用resample对数据分组，之后再调用聚合函数

In [ ]:
ts.resample("M", kind="period").mean()    #注意区分这个与上一个

__向下采样__

In [2]:
import numpy as np 
import pandas as pd
rng = pd.date_range('2000-01-01', periods=12, freq='T')
ts = pd.Series(np.arange(12), index=rng)

ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, dtype: int32

In [10]:
ts.resample('7min').sum()

2000-01-01 00:00:00    21
2000-01-01 00:07:00    45
Freq: 7T, dtype: int32

In [11]:
ts.resample('7min', closed='right').sum()

1999-12-31 23:53:00     0
2000-01-01 00:00:00    28
2000-01-01 00:07:00    38
Freq: 7T, dtype: int32

上面这个地方一定要注意区分

In [ ]:
ts.resample('5min').ohlc()

__向上采样与插值__

In [ ]:
#frame是每周数据，索引是DatetimeIndex
df_daily = frame.resample("D").asfreq()

In [ ]:
frame.resample("D").ffill()

__使用区间进行重新采样__

对以区间为索引的数据进行采样与时间戳的情况类似  
由于区间涉及时间范围，向上采样和向下采样就更为严格  
* 在向下采样中，目标频率必须是原频率的子区间
* 在向上采样中，目标频率必须是原频率的父区间

In [ ]:
frame = pd.DataFrame(np.random.randn(24, 4), index=pd.PeriodIndex('1-2000', '12-2001', freq="M"), columns=['','','',''])
annual_frame = frame.resample('A-DEC').mean()

##  7.  移动窗口函数

与其他的统计函数类似，这些函数会自动排除缺失数据  
rolling算子，它的行为与resample和groupby类似。rolling可以在Series或DataFrame上通过一个window进行调用

In [ ]:
close_px.AAPL.rolling(250).mean()    #也接收表示固定大小的时间偏执字符串，而不只是一个区间的集合数字。('20D')

appl_std250 = close_px.AAPL.rolling(250, min_periods=10).std()    #△

In [ ]:
#为了计算扩展窗口均值，使用expanding算子，而不是rolling。扩展均值从时间序列的起始位置开始时间窗口，并增加窗口的大小，知道它覆盖整个序列。
expanding_mean = appl_std250.expanding().mean()

In [ ]:
#在DataFrame上调用一个移动窗口函数会将变换应用到每一列上
close_px.rolling(60).mean()

__指数加权函数__

指定一个常数衰减因子以向更多近期观测值提供更多权重，可以替代使用具有相等加权观察值的静态窗口尺寸的方法。

In [ ]:
aapl_px.ewm(span=30).mean()

__二元移动窗口函数__

金融分析师经常对股票与基准指数的关联性感兴趣

In [ ]:
corr = returns.rolling(125, min_periods=100).corr(spx_rets)

__用户自定义的移动窗口函数__

In [ ]:
returns.AAPL.rolling(250).apply(score_at_2percent)    #唯一的要求是该函数从每个数组中产生一个单值（缩聚）

本章完